In [5]:
import os
import numpy as np
import pandas as pd
from scipy.special import logsumexp
from sklearn.decomposition import PCA
import gzip

eps = 0.1  # small epsilon value


class GaussianMixtureModel:
    def __init__(self, num_components=5, max_iter=500, tol=1e-5):
        # khởi tạo GMM với các tham số
        self.num_components = num_components
        self.max_iter = max_iter
        self.tol = tol

    def fit(self, X):
        n, d = X.shape
        self.log_likelihood_loss = {}
        log_r_matrix = np.eye(self.num_components)
        log_r_matrix = log_r_matrix[np.random.choice(
            self.num_components, size=n)].T
        iter_num = 0

        for it in range(self.max_iter):
            # M-Step
            r_k = np.sum(log_r_matrix, axis=1) + \
                (10 * np.finfo(float).eps)
            pi = r_k / n
            means = log_r_matrix @ X / r_k[:, np.newaxis]  # (K, d)
            variances = log_r_matrix @ (X ** 2) / r_k[:, np.newaxis]
            variances -= means ** 2
            variances += eps  # Add epsilon to variances

            # E-Step
            sum_log_r, log_r_matrix = self._multivariate_gaussian(
                X, pi, variances, means)
            log_r_matrix = np.exp(log_r_matrix - sum_log_r)

            # compute loss
            self.log_likelihood_loss[it] = -np.sum(sum_log_r)
            loss_difference = 0
            if it > 1:
                loss_difference = np.abs(self.log_likelihood_loss[it] -
                                         self.log_likelihood_loss[it - 1]) / (np.abs(self.log_likelihood_loss[it]) + eps)
                if loss_difference <= self.tol:
                    iter_num = it
                    break

        print("EM for GMM converged after ", iter_num + 1,
              "iteration, with loss: ", self.log_likelihood_loss[iter_num])
        self.params = {'log_r_matrix': log_r_matrix,
                       'means': means, 'variances': variances, 'pi': pi}

    def predict_proba(self, X):
        class_probab_list = np.zeros((self.num_components, len(X)))
        for digit in range(self.num_components):
            sum_log_r, r = self._multivariate_gaussian(
                X, self.params['pi'], self.params['variances'], self.params['means'])
            class_probab_list[digit] = logsumexp(
                r, axis=0) * (X.shape[0] / len(X))
        return class_probab_list

    def _multivariate_gaussian(self, X, pi, variances, means):
        reversed_var = 1 / (variances + eps)  # Add epsilon to variances
        log_r_matrix = (X ** 2) @ reversed_var.T
        log_r_matrix -= 2 * X @ (means * reversed_var).T
        log_r_matrix[:] += np.sum(
            (means ** 2) * reversed_var, axis=1)
        log_r_matrix *= -0.5
        # Add epsilon to variances
        log_r_matrix[:] += np.log(pi) - 0.5 * \
            np.sum(np.log(variances + eps), axis=1)
        log_r_matrix = log_r_matrix.T
        sum_log_r = logsumexp(log_r_matrix, axis=0)
        return sum_log_r, log_r_matrix
  


In [6]:
data_path = '' # có thể thay đổi

# Đường dẫn
images_train_path = os.path.join(data_path, 'train-images-idx3-ubyte.gz')
labels_train_path = os.path.join(data_path, 'train-labels-idx1-ubyte.gz')

images_test_path = os.path.join(data_path, 't10k-images-idx3-ubyte.gz')
labels_test_path = os.path.join(data_path, 't10k-labels-idx1-ubyte.gz')


def get_mnist_data_as_dataframe(images_path, labels_path, shuffle=False, image_size=28):
    # Đọc dữ liệu ảnh
    with gzip.open(images_path, 'r') as f_images:
        # Bỏ qua 16 byte đầu tiên vì đây không phải là dữ liệu, chỉ là thông tin header
        f_images.read(16)

        # Đọc tất cả dữ liệu sau khi bỏ đi phần head
        buf_images = f_images.read()

        # Chuyển dữ liệu thành numpy array và đổi dtype thành float32
        images = np.frombuffer(buf_images, dtype=np.uint8).astype(np.float32)

        # Reshape dữ liệu thành dạng (num_images, image_size*image_size)
        images = images.reshape(-1, image_size * image_size)

    # Đọc tệp labels
    with gzip.open(labels_path, 'r') as f_labels:
        f_labels.read(8)
        buf_labels = f_labels.read()
        labels = np.frombuffer(buf_labels, dtype=np.uint8).astype(np.int64)

    # Tạo DataFrame từ dữ liệu ảnh
    df_images = pd.DataFrame(images)

    # Thêm cột label vào DataFrame dữ liệu ảnh
    df_images['label'] = labels

    # Trộn dữ liệu trong dataframe
    if shuffle:
        df_images = df_images.sample(frac=1).reset_index(drop=True)

    return df_images

# dataframe train
mnist_train_df = get_mnist_data_as_dataframe(
    images_train_path, labels_train_path, shuffle=True)

# dataframe test
mnist_test_df = get_mnist_data_as_dataframe(
    images_test_path, labels_test_path, shuffle=True)

# Convert DataFrames to NumPy arrays
y_train = mnist_train_df['label'].values
X_train = mnist_train_df.drop(columns=['label']).values / 255.0

y_test = mnist_test_df['label'].values
X_test = mnist_test_df.drop(columns=['label']).values / 255.0


In [7]:
# PCA dimensionality reduction
effective_features = 50   # number of top effective features to be selected by PCA
pca = PCA(n_components=effective_features)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Gaussian Mixture Model (GMM) with EM algorithm
gm_num = 10  # number of Gaussian Models in each GMM
GMMs = {}
log_likelihood_loss = {}
for digit in range(10):  # assuming there are 10 digits (0-9)
    print('Fitting GMM to digit', digit)
    X_digit_train = X_train_pca[y_train == digit]
    GMMs[digit] = GaussianMixtureModel(num_components=gm_num)
    GMMs[digit].fit(X_digit_train)
    print('GMM parameters computed for digit =', digit)

# Predictions
class_probab_list = np.zeros((10, len(X_test_pca)))
for digit in range(10):
    class_probab_list[digit] = GMMs[digit].predict_proba(X_test_pca)[digit]

predictions = np.argmax(class_probab_list, axis=0)
accuracy = np.mean(predictions == y_test)
print("Accuracy:", accuracy)

Fitting GMM to digit 0
EM for GMM converged after  44 iteration, with loss:  25467.13459861849
GMM parameters computed for digit = 0
Fitting GMM to digit 1
EM for GMM converged after  50 iteration, with loss:  -128512.8600072753
GMM parameters computed for digit = 1
Fitting GMM to digit 2
EM for GMM converged after  77 iteration, with loss:  51231.603681718225
GMM parameters computed for digit = 2
Fitting GMM to digit 3
EM for GMM converged after  69 iteration, with loss:  38195.42047661245
GMM parameters computed for digit = 3
Fitting GMM to digit 4
EM for GMM converged after  94 iteration, with loss:  19821.864729135086
GMM parameters computed for digit = 4
Fitting GMM to digit 5
EM for GMM converged after  78 iteration, with loss:  31788.69454777378
GMM parameters computed for digit = 5
Fitting GMM to digit 6
EM for GMM converged after  34 iteration, with loss:  12525.670616518608
GMM parameters computed for digit = 6
Fitting GMM to digit 7
EM for GMM converged after  227 iteration,

In [11]:
from sklearn.metrics import davies_bouldin_score

# Calculate Davies-Bouldin Index
davies_bouldin_idx = davies_bouldin_score(X_test_pca, predictions)
print("Davies-Bouldin Index:", davies_bouldin_idx)


Davies-Bouldin Index: 3.282100401206958
